In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Traditional RAG"

from langchain_community.document_loaders import TextLoader

loader=TextLoader("blog1.txt")
docs=loader.load()
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

split_docs=splitter.split_documents(docs)
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
import faiss
from langchain_community.vectorstores import FAISS

index=faiss.IndexFlatL2(1536)
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index=faiss.IndexFlatL2(1536)
vectorstore = FAISS(
    embeddings, 
    index, 
    docstore=InMemoryDocstore({}), 
    index_to_docstore_id={})

vectorstore.add_documents(documents=split_docs)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the information provided in the context below. "
    "If the answer is not contained in the context, respond with 'I don't know.'\n\n"
    "Context:\n{context}\n\nQuestion:\n{question}"
)

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0
)

def join_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)


question = "What is the main topic of the blog?"
result = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {result}")


Question: What is the main topic of the blog?
Answer: The main topic of the blog is ensuring your cloud setup can handle pressure, specifically in preparation for Black Friday.
